In [3]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import torch
from notebooks.experiment_robust import load_robust_experiment
from notebooks.experiment_torch import load_torchvision_experiment

model, dl_train, dl_eval = load_robust_experiment("Salman2020Do_R50", "imagenet", batch_size=128)

Creating Dataset:   0%|          | 0/25000 [00:00<?, ?it/s]

Creating Dataset:   0%|          | 0/10000 [00:00<?, ?it/s]

Extracting Samples:   0%|          | 0/25 [00:00<?, ?it/s]

Extracting Samples:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/196 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB. GPU 0 has a total capacity of 47.33 GiB of which 39.56 MiB is free. Process 3448948 has 9.91 GiB memory in use. Process 3458626 has 17.28 GiB memory in use. Process 3472266 has 18.35 GiB memory in use. Including non-PyTorch memory, this process has 1.73 GiB memory in use. Of the allocated memory 1.34 GiB is allocated by PyTorch, and 84.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model

In [ ]:
import torchattacks

attk_eps = 8 / 255
attk_alpha = 8 / 255
attk_steps = 5

# torch_attack = torchattacks.FGSM(model, eps=attk_eps) # very fast; good
# torch_attack = torchattacks.FFGSM(model, eps=attk_eps, alpha=attk_alpha)  # very fast; very good
# torch_attack = torchattacks.DIFGSM(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # fast
# torch_attack = torchattacks.RFGSM(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # fast; very good
# torch_attack = torchattacks.BIM(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # fast
# torch_attack = torchattacks.TIFGSM(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # fast
# torch_attack = torchattacks.MIFGSM(model, eps=attk_eps, alpha=attl_alpha, steps=attk_steps)  # fast
# torch_attack = torchattacks.NIFGSM(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # fast
# torch_attack = torchattacks.PGDL2(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # fast
# torch_attack = torchattacks.PGD(model) # fast
# torch_attack = torchattacks.TPGD(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # fast
# torch_attack = torchattacks.UPGD(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps, loss="dlr") # fast; very good
# torch_attack = torchattacks.APGD(model, eps=attk_eps, steps=attk_steps, loss="dlr") # fast; good
# torch_attack = torchattacks.Jitter(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # fast; good
# torch_attack = torchattacks.PIFGSM(model, max_epsilon=attk_eps, num_iter_set=attk_steps) # normal
# torch_attack = torchattacks.PIFGSMPP(model, max_epsilon=attk_eps, num_iter_set=attk_steps) # normal
# torch_attack = torchattacks.PGDRS(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # normal
# torch_attack = torchattacks.EOTPGD(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # normal
# torch_attack = torchattacks.PGDRSL2(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # slow
# torch_attack = torchattacks.SINIFGSM(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # slow
# torch_attack = torchattacks.VMIFGSM(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # very slow
# torch_attack = torchattacks.EADEN(model, lr=attk_alpha, max_iterations=attk_steps) # very slow
# torch_attack = torchattacks.VNIFGSM(model, eps=attk_eps, alpha=attk_alpha, steps=attk_steps) # very slow
# torch_attack = torchattacks.APGDT(model, eps=attk_eps, steps=attk_steps) # very slow
# torch_attack = torchattacks.SPSA(model, eps=attk_eps, lr=attk_alpha) # very slow
# torch_attack = torchattacks.FAB(model, eps=attk_eps, steps=attk_steps) # very slow
# torch_attack = torchattacks.CW(model, steps=attk_steps, lr=attk_alpha) # very slow
# torch_attack = torchattacks.AutoAttack(model, eps=attk_eps) # super slow
# torch_attack = torchattacks.Square(model, eps=attk_eps) # super slow
# torch_attack = torchattacks.DeepFool(model, steps=attk_steps) # super slow

In [ ]:
import torchattacks.attack
from ulib.attack import StopCriteria
from ulib.attacks.iml_uap_sched import IML_UAP_SCHED
from ulib.attacks.iml_uap import CosSim, L2Diff, L1Diff
from ulib.pert_module import PertModule
from ulib.activation_extractor import ActivationExtractor

stop = StopCriteria(max_epochs=30, max_time=60 * 30)
pert_model = PertModule(model, data_shape=dl_train.get_tensor(0).shape[1:], eps=16 / 255, random_init=True)
optim = torch.optim.Adam(pert_model.parameters(), lr=2e-2)
criterion = CosSim(aggr_fn=lambda losses: torch.prod(losses, dim=-1))
activ_extractor = ActivationExtractor(model, "model.layer2", exact_match=True)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim, T_0=50, T_mult=1, eta_min=5e-5)
grad_scaler = torch.GradScaler(device=pert_model.device.type)


def attack_builder(model: torch.nn.Module, epoch: int) -> torchattacks.attack.Attack:
    eps = (pert_model.eps / stop.max_epochs) * (epoch + 1)
    # return torchattacks.FFGSM(model, eps=eps, alpha=eps)
    # return torchattacks.PGD(model, eps=eps, alpha=eps / 3, steps=5)
    return torchattacks.CW(model, steps=5, lr=eps / 3)


attack = IML_UAP_SCHED(
    pert_model=pert_model,
    optimizer=optim,
    attack_builder_func=attack_builder,
    criterion=criterion,
    activ_extractor=activ_extractor,
    # scheduler=scheduler,
    grad_scaler=grad_scaler,
    skip_already_fooled=True,
    skip_failed_attacks=True,
    log_dir="logs",
)

pert = attack.fit(dl_train, dl_eval, stop)

In [ ]:
attack.close()

In [ ]:
from ulib import eval

eval.full_analysis(pert_model, dl_eval)